<a href="https://colab.research.google.com/github/arkigos/craigsboat/blob/master/boatz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread -U
!pip install df2gspread


  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1
     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for df2gspread: filename=df2gspread-1.0.4-cp36-none-any.whl size=11953 sha256=998b49051f40ae3abcd84c4f47745bf3326febe9353d56d166729d6606a2f971
  Stored in directory: /root/.cache/pip/wheels/19/d2/15/07b0a45a18edadbf0bb3a5655e997925e2b9cbf08b4a95342e
Successfully built df2gspread
ERROR: firebase-admin 4.1.0 has requirement google-api-python-client>=1.7.8, but you'll have google-api-python-client 1.6.7 which is incompatible.
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [18]:
from bs4 import BeautifulSoup 
import requests, urllib3
from IPython.display import Image, display
from IPython.core.display import HTML 
import re, collections, itertools
import pandas
#rom df2gspread import df2gspread as d2g
#from df2gspread import gspread2df as g2d


In [19]:
import os

os.chdir('/content/drive/My Drive/boatz')

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

#fun = GoogleCredentials.get_application_default()

#fun.to_json()

#ecc = g2d.gspread.authorize(GoogleCredentials.get_application_default())

#g2d.download('west_coast')

In [20]:
max_price = 15000
min_boat_length_overall = 33
max_boat_length_overall = 47

main_url = urllib3.util.parse_url('https://losangeles.craigslist.org/search/sss?query=boat&sort=rel&max_price=' + str(max_price) + '&min_boat_length_overall=' + str(min_boat_length_overall) + '&max_boat_length_overall=' + str(max_boat_length_overall) + '&boat_propulsion_type=1')

location_list = { 
    'west_coast': ('honolulu','sfbay','humboldt','oregoncoast','portland','seattle','olympic','victoria','skagit','nanaimo','bellingham','vancouver','abbotsford','sunshine','comoxvalley','monterey','slo','santamaria','santabarbara','ventura','orangecounty','sandiego','tijuana','bajasur','hermosillo','mendocino','losangeles')
   ,'east_coast_caribbean': ('yucatan','veracruz','brownsville','mcallen','corpuschristi','victoriatx','houston','galveston','beaumont','lakecharles','houma','lafayette','neworleans','batonrouge','gulfport','pensacola','mobile','okaloosa','panamacity','tallahassee','gainesville','ocala','lakecity','tampa','lakeland','sarasota','cfl','keys','fortmyers','miami','spacecoast','treasure','daytona','staugustine','jacksonville','brunswick','savannah','hiltonhead','charleston','myrtlebeach','wilmington','onslow','eastnc','outerbanks','norfolk','richmond','fredericksburg','smd','washingtondc','easternshore','annapolis','baltimore','delaware','southjersey','orlando','caribbean','santodomingo','puertorico','virgin')
}

def all_boats(soup):
  list_boats = []
  for each in soup.findAll('li', {'class': 'result-row'}):
    title = each.find('p', {'class': 'result-info'}).a.text
    price = each.find('span', {'class': 'result-price'}).text.strip('$')
    link = each.a.attrs['href']
    images = []
    attrs = collections.defaultdict()
    if 'data-ids' in each.a.attrs:
      images = re.findall(':([^,]+),',each.a.attrs['data-ids'])
      images = ['https://images.craiglist.org/' + image + '_300x300.jpg' for image in images]
    
    boat_page = BeautifulSoup(requests.get(each.a.attrs['href']).text)
    
    attrgroup = boat_page.find('p', {'class': 'attrgroup'})
    if attrgroup:
      for attr in attrgroup.findAll('span'):
        if ':' not in attr.text:
          k, v = attr.text, ''
        else:
          k, v = attr.text.split(':')
        attrs[k.strip()] = v.strip()
    
   
    list_boats.append({'title': title
                      ,'price': price
                      , 'link': link
                      , 'images': images
                      , 'attrs': attrs
                       })
    
  return list_boats




In [ ]:
for loc in location_list:
  boats = [ all_boats(BeautifulSoup(requests.get('https://' + craig + '.craigslist.org' + main_url.request_uri).text)) for craig in location_list[loc] ]

  
  df = pandas.DataFrame([ x for x in itertools.chain.from_iterable(boats) ])
  
  data = pandas.concat([df.drop(['attrs'], axis=1), df['attrs'].apply(pandas.Series)], axis=1)
  data.sort_values("link", inplace=True)
  data.drop_duplicates(subset="link", inplace=True)
  data.fillna('', inplace=True)
  data.replace('\n', '')
  ecc = gc.open(loc)
  if ecc.sheet1.get_all_values() == []:
    ecc.sheet1.update([data.columns.values.tolist()] + data.values.tolist())

  cur_list = pandas.DataFrame(ecc.sheet1.get_all_records())
  
  data.merge(cur_list, on='link', suffixes=('', '_old'))

  ecc.sheet1.update([data.columns.values.tolist()] + data.values.tolist())

  

In [23]:
BeautifulSoup(requests.get('https://' + 'sfbay' + '.craigslist.org' + main_url.request_uri).text)


<html><body><p>This IP has been automatically blocked.
If you have questions, please email: blocks-b1599160574223189@craigslist.org
</p></body></html>

In [21]:
  for loc in location_list:
    boats = [ all_boats(BeautifulSoup(requests.get('https://' + craig + '.craigslist.org' + main_url.request_uri).text)) for craig in location_list[loc] ]

  
    df = pandas.DataFrame([ x for x in itertools.chain.from_iterable(boats) ])
    
    data = pandas.concat([df.drop(['attrs'], axis=1), df['attrs'].apply(pandas.Series)], axis=1)
    data.sort_values("link", inplace=True)
    data.drop_duplicates(subset="link", inplace=True)
    data.drop_duplicates(subset='title', inplace=True)
    data.fillna('', inplace=True)
    
    #ecc = g2d.download('west_coast')


    data.to_csv(loc)

KeyError: ignored

In [ ]:
main_url.request_uri

'/search/sss?query=boat&sort=rel&max_price=15000&min_boat_length_overall=33&max_boat_length_overall=47&boat_propulsion_type=1'

In [ ]:
boats = [ all_boats(BeautifulSoup(requests.get('https://' + craig + '.craigslist.org' + main_url.request_uri).text)) for craig in location_list['west_coast'] ]


df = pandas.DataFrame([ x for x in itertools.chain.from_iterable(boats) ])
data = pandas.concat([df.drop(['attrs'], axis=1), df['attrs'].apply(pandas.Series)], axis=1)
data.sort_values("link", inplace=True)
data.drop_duplicates(subset="link", inplace=True)
data.drop_duplicates(subset='title', inplace=True)
data.fillna('', inplace=True)

#ecc = g2d.download('west_coast')


data.to_csv('west_coast')

KeyError: ignored